# Synchronized tdmclient interactive session

This notebook borrows largely from the repl help file of tdmclient.

First, make sure that tdmclient is installed for Jupyter:

In [ ]:
%pip install --upgrade --quiet tdmclient

Import the required class.

In [ ]:
import tdmclient.notebook

Connect to the TDM and start variable synchronization with the first robot:

In [ ]:
await tdmclient.notebook.start()

Read a Thymio variable:

In [ ]:
temp = temperature
print(temp)

Change a variable:

In [ ]:
leds_top = [0,32,32]

Write a program to be run on the Thymio:

In [ ]:
on = False
timer_period[0] = 500
@onevent
def timer0():
    global on, leds_top
    on = not on
    if on:
        leds_top = [32, 32, 0]
    else:
        leds_top = [0, 0, 0]

Run it. Actually it is not just the previous cell which is compiled to something the Thymio can execute, but all the functions decorated with `@onevent`, the functions they call, the Thymio variables which have been set, and other global variables they use.

In [ ]:
run()

Stop it:

In [ ]:
stop()

You can also run on the Thymio a Python program contained in a single cell:

In [ ]:
%%run_python
v = [32, 0, 32, 0, 32, 0, 32, 0]
leds_circle = v

Or an Aseba program:

In [ ]:
%%run_aseba
var v[] = [32, 32, 32, 0, 0, 0, 32, 32]
leds.circle = v

The Python program is converted to Aseba (_transpiled_) before being compiled and executed on the Thymio. You can see the intermediate Aseba code:

In [ ]:
%%transpile_to_aseba
v = 4 * [32, 0]
leds_circle = v

It's also possible to use `print` statements in Python programs running on the Thymio. They're converted to events: the Thymio sends the numeric values, which can be the result of any expressions, and the notebook on the computer receives them and combines them with constant string arguments of `print` and displays the result.

Instead of just `%%run_python`, the program cell must begin with `%%run_python --wait` in order to run as long as required to process events. To stop it, either call `exit()` in the program, or click the Stop button of Jupyter (_interrupt the kernel_).

In [ ]:
%%run_python --wait

i = 0

timer_period[0] = 500

@onevent
def timer0():
    global i, leds_top
    i += 1
    is_odd = i % 2 == 1
    if is_odd:
        print(i, "odd")
        leds_top = [0, 32, 32]
    else:
        print(i, "even")
        leds_top = [0, 0, 0]
    if i >= 10:
        exit()

By default, magic commands `%%run_python` and `%%transpile_to_aseba` define automatically all the variables, constants and functions specific to the Thymio. They work as if the code starts with `from thymio import *`. For more flexibility, you can suppress these definitions with the `--nothymio` option and load the `thymio` module exactly the way you want: as a whole, or only some symbols, or with aliases, or in the scope of a function definition.

In [ ]:
%%transpile_to_aseba --nothymio
import thymio
thymio.leds_top = thymio.GREEN

In [ ]:
%%transpile_to_aseba --nothymio
from thymio import leds_top as color_led, MAGENTA
color_led = MAGENTA

## Events between the notebook and the robot

To retrieve data from the robot and process them further in your notebook, you can send events with `emit`. In the program below, we collects 20 samples of the front proximity sensor, one every 200ms (5 per second), i.e. during 4 seconds.

In [ ]:
%%run_python --wait

i = 0
timer_period[0] = 200

@onevent
def timer0():
    global i, prox_horizontal
    i += 1
    if i > 20:
        exit()
    emit("front", prox_horizontal[2])

Events received by the computer are collected automatically. We retrieve them with `get_event_data(event_name)`, a list of all the data sent by `emit`, which are lists themselves. 

In [ ]:
data = get_event_data("front")
print(data)

You can send events with different names. You can also reset an event collection by calling `clear_event_data(event_name)`, or without argument to clear all the events:

In [ ]:
clear_event_data()

Instead of calling `clear_event_data()` without argument, the option `--clear-event-data` of the magic command `%%run_python` has the same effect and avoids to evaluate a separate notebook cell.

You can also send events in the other direction, from the notebook to the robot. This can be useful for instance if you implement a low-level behavior on the robot, such as obstacle avoidance and sensor acquisition, and send at a lower rate high-level commands which require more computing power available only on the PC.

The Thymio program below listens for events named `color` and changes the top RGB led color based on a single number. Bits 0, 1 and 2 represent the red, green, and blue components respectively.

In [ ]:
%%run_python

@onevent
def color(c):
    global leds_top
    leds_top[0] = 32 if c & 1 else 0
    leds_top[1] = 32 if c & 2 else 0
    leds_top[2] = 32 if c & 4 else 0

Now that the program runs on the robot, we can send it `color` events. The number of values in `send_event` should match the `@onevent` declaration. They can be passed as numeric arguments or as arrays.

In [ ]:
for col in range(8):
    send_event("color", col)
    sleep(0.5)

### Module `clock`

The module `clock` provides functions to get the current time since the start of the program or the last call to its function `reset()`. It can be used to measure the time between two events or to add time information to data sent from the robot to the PC with events.

In the following program, the left button displays the time in seconds, the right button in 1/50 seconds, the backward button resets the clock, and the center button exits the program.

In [ ]:
%%run_python --wait

import clock

@onevent
def button_left():
    print("clock.seconds()", clock.seconds())

@onevent
def button_right():
    print("clock.ticks_50Hz()", clock.ticks_50Hz())

@onevent
def button_backward():
    print("clock.reset()")
    clock.reset()

@onevent
def button_center():
    exit()

The values are based on a counter incremented 50 times per second. Since it's stored in a signed 16-bit integer, like all Thymio variables, there is an overflow after 32767/50 seconds, or 5 minutes and 55 seconds. If your program runs longer, use the clock to measure smaller intervals and reset it for each new interval.